# Microbit Probing

In [5]:
import asyncio
import logging
import traceback
import time

from bleak import BleakClient

NOT_FOUND_LABEL = "NOT FOUND"
UNDEFINED_LABEL = "UNDEFINED"

address = NOT_FOUND_LABEL
uuid = UNDEFINED_LABEL
model = UNDEFINED_LABEL
bdate = UNDEFINED_LABEL 

address = ('ED:B9:4A:79:34:B4')

client = BleakClient(address)

counter = 0
while( not client.is_connected):
    await client.connect()
    counter += 1
    if client.is_connected:
        print('client connected')
        break
    if (counter >= 1000):
        print('timed out')
        break


client connected


In [6]:
await client.pair()

True

In [7]:
for service in client.services:
    print(service)

00001800-0000-1000-8000-00805f9b34fb (Handle: 1): Generic Access Profile
00001801-0000-1000-8000-00805f9b34fb (Handle: 10): Generic Attribute Profile
0000fe59-0000-1000-8000-00805f9b34fb (Handle: 14): Nordic Semiconductor ASA
e97dd91d-251d-470a-a062-fa1922dfa9a8 (Handle: 18): Unknown
0000180a-0000-1000-8000-00805f9b34fb (Handle: 22): Device Information
e95d93af-251d-470a-a062-fa1922dfa9a8 (Handle: 29): MicroBit Event Service


In [8]:
for service in client.services:
    print(f"[Service] {service.uuid}: {service.description}")
    for char in service.characteristics:
        characteristic = char
        try:
            uuid = bytes(await client.read_gatt_char(char.uuid))
        except:
            print(f'EXCEPTION:{traceback.format_exception}')

        properties = char.properties
        desc = char.description
        if "read" in char.properties:
            try:
                value = bytes(await client.read_gatt_char(char.uuid))
                print(f"{char.description} - {char.uuid} -> {value}")
            except Exception as e:
                value = str(e).encode()
        else:
            value = None
        print(f"\t[Characteristic] {char.uuid}: ({','.join(char.properties)}) | Name: {char.description}, Value: {value} ")
        for descriptor in char.descriptors:
            value = await client.read_gatt_descriptor(descriptor.handle)
            print(f"\t\t[Descriptor] {descriptor.uuid}: (Handle: {descriptor.handle}) | Value: {bytes(value)} ")

[Service] 00001800-0000-1000-8000-00805f9b34fb: Generic Access Profile
Device Name - 00002a00-0000-1000-8000-00805f9b34fb -> b'BBC micro:bit [zugeg]'
	[Characteristic] 00002a00-0000-1000-8000-00805f9b34fb: (read) | Name: Device Name, Value: b'BBC micro:bit [zugeg]' 
Appearance - 00002a01-0000-1000-8000-00805f9b34fb -> b'\x00\x00'
	[Characteristic] 00002a01-0000-1000-8000-00805f9b34fb: (read) | Name: Appearance, Value: b'\x00\x00' 
Peripheral Preferred Connection Parameters - 00002a04-0000-1000-8000-00805f9b34fb -> b'\x08\x00\x10\x00\x00\x00\x90\x01'
	[Characteristic] 00002a04-0000-1000-8000-00805f9b34fb: (read) | Name: Peripheral Preferred Connection Parameters, Value: b'\x08\x00\x10\x00\x00\x00\x90\x01' 
Central Address Resolution - 00002aa6-0000-1000-8000-00805f9b34fb -> b'\x01'
	[Characteristic] 00002aa6-0000-1000-8000-00805f9b34fb: (read) | Name: Central Address Resolution, Value: b'\x01' 
[Service] 00001801-0000-1000-8000-00805f9b34fb: Generic Attribute Profile
EXCEPTION:<function

OSError: [WinError -2147483629] The object has been closed

In [3]:
await client.disconnect()

True

In [4]:
client.is_connected

False